# Imports

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
possibilidades=[[0,0,0],[0,0,1],[0,0,2],
                [0,1,0],[0,1,1],[0,1,2],
                [1,0,0],[1,0,1],[1,0,2],
                [1,1,0],[1,1,1],[1,1,2],
                [2,0,0],[2,0,1],[2,0,2],
                [2,1,0],[2,1,1],[2,1,2],
                [3,0,0],[3,0,1],[3,0,2],
                [3,1,0],[3,1,1],[3,1,2]]

df = pd.read_csv("doencas.csv")
df_externo = pd.read_csv("background.csv")
for index in df.index:
    df.at[index,"id"] = -1

# Defining maximum hierarchy and Precision

In [3]:
def max_hye(index):
    if (index== 0): return 3
    elif (index == 1): return 1
    else: return 2


def precision(df,lista):
    summ =  0
    for i in df.index:
        for index in [0,1]:
                summ += lista[index]/max_hye(index)
        if(df.at[i,"estado"]=="*"): summ+=1
        elif(df.at[i,"cidade"]=="*"): summ+=1/2
        else: summ+=0
            
    return 1 - summ/(df.shape[0] * 3)

In [4]:
def count_equivl(df): #retorna a quantidade de elementos de cada classe de equivalencia
    return df.groupby(["estado","cidade","genero","data"], axis=0, level=None, as_index=False).count()

def classEquivalence(df, b,c,d,e): #retorna a classe de equivalencia com os paremetros passados
    return df[(df["estado"] == b) & (df["cidade"] == c) & (df["genero"] == d) & (df["data"] == e)]

# Anonymizing The Dataset

In [5]:
def anonimiza(df, lista):
    for index in df.index:

        if(lista[0] == 0): pass
        elif (lista[0] == 1):
            df.at[index, "data"] = "*" + str(df.at[index,"data"][2:8])
        elif (lista[0] == 2):
            df.at[index, "data"] = "*/*" + str(df.at[index,"data"][5:8])
        else:
            df.at[index, "data"] = "*/*/*"

        if(lista[1] == 0): pass
        else:
            df.at[index, "genero"] = "*"
            
        if(lista[2] == 0): pass
        elif (lista[2] == 1):
            df.at[index, "cidade"] = "*"
        else:
            df.at[index, "cidade"] = "*"
            df.at[index, "estado"] = "*"

# Delta Presence

In [6]:
def delta(df, df_externo, delmin, delmax):
    
    for p in possibilidades:
        
        df_copy = df.copy()
        df_externo_copy = df_externo.copy()
        deltas_calculados = []
        
        anonimiza(df_copy, p)
        anonimiza(df_externo_copy,p)
        
        classes = count_equivl(df_copy)
        for index in classes.index:
            b = classes.at[index,"estado"]
            c = classes.at[index,"cidade"]
            d = classes.at[index,"genero"]
            e = classes.at[index,"data"]
            equivClassDF_copy = classEquivalence(df_copy,b,c,d,e)
            equivClassDF_externo = classEquivalence(df_externo_copy,b,c,d,e)
            if(equivClassDF_externo.shape[0] == 0): return print("divisao por zero")
            deltas_calculados.append(equivClassDF_copy.shape[0]/equivClassDF_externo.shape[0])
            
            
        if(min(deltas_calculados)>= delmin and max(deltas_calculados)<=delmax): return [df_copy, p]
    print("Erro")
    return [df,[0,0,0]]
        

# Precision for delta min = 10% and delta max = 50 %

In [7]:
l = delta(df,df_externo, 0.1,0.5)
precision(l[0],l[1])
print(precision(l[0],l[1]))
data = pd.DataFrame(l[0]) 
data.to_csv("doencas_40.csv",index = False,sep=",")

0.5555555555555576


# Precision for delta min = 10% and delta max = 40 %

In [8]:
l = delta(df,df_externo, 0.1,0.4)
precision(l[0],l[1])
print(precision(l[0],l[1]))
data = pd.DataFrame(l[0]) 
data.to_csv("doencas_30.csv",index = False,sep=",")

0.5555555555555576


# Precision for delta min = 10% and delta max = 30 %

In [9]:
l = delta(df,df_externo, 0.1,0.3)
precision(l[0],l[1])
print(precision(l[0],l[1]))
data = pd.DataFrame(l[0]) 
data.to_csv("doencas_20.csv",index = False,sep=",")

0.5555555555555576


# Precision for delta min = 10% and delta max = 20 %

In [10]:
l = delta(df,df_externo, 0.1,0.2)
precision(l[0],l[1])
print(precision(l[0],l[1]))
data = pd.DataFrame(l[0]) 
data.to_csv("doencas_10.csv",index = False,sep=",")

0.0
